In [3]:
import pandas as pd
import os

path = "/home/rupnik/macocu/task8/voxpopuli/hr/"
all_oggs =  []
for root, dirs, files in os.walk(path):
    for name in files:
        if name.endswith(".ogg"):
            all_oggs.append(name)



def has_diacritics(s):
    # import re
    # pattern = "[šđčćž]"
    # p = re.compile(pattern)
    # try:
    #     return p.search(s.lower()) is not None
    # except:
    #     return False
    return isinstance(s, str)
def assign_path(s):
    year = s[0:4]
    return os.path.join(
        path,
        year,
        s+".ogg"
    )

train = pd.read_csv("voxpopuli/hr/asr_train.tsv", sep="\t")
train["filename"] = train.id + ".ogg"
train["raw_has_diacritics"] = train.raw_text.apply(has_diacritics)
train["path"] = train.id.apply(assign_path)

test = pd.read_csv("voxpopuli/hr/asr_test.tsv", sep="\t")
test["filename"] = test.id + ".ogg"
test["raw_has_diacritics"] = test.raw_text.apply(has_diacritics)
test["path"] = test.id.apply(assign_path)

dev = pd.read_csv("voxpopuli/hr/asr_dev.tsv", sep="\t")
dev["filename"] = dev.id + ".ogg"
dev["raw_has_diacritics"] = dev.raw_text.apply(has_diacritics)
dev["path"] = dev.id.apply(assign_path)


df = pd.concat([train, dev, test], ignore_index=True)
df = df[~df.raw_text.isna()]

In [5]:
from pydub import AudioSegment
def get_duration(path):
    segment = AudioSegment.from_ogg(path)
    return len(segment)/1000

from concurrent.futures import ProcessPoolExecutor
with ProcessPoolExecutor(max_workers=60) as executor:
    durs = list(executor.map(get_duration, df.path))

In [6]:
sum(durs) / 3600

21.542951111111144